Importing data

In [5]:
# uncomment if using this code in RStudio
# setwd(dirname(rstudioapi::getActiveDocumentContext()$path))

library(coin)
library(lme4)
library(lmerTest)
library(plyr)
library(xtable)
library(LMERConvenienceFunctions)
library(car)
# library(lmerTest) see also https://link.springer.com/article/10.3758/s13428-016-0809-y 

source("r_utils/mer-utils.R")
source("r_utils/regression-utils.R")

data_path = "C:/Users/Arkady/Google Drive/data/CoM_fixed_duration/processed/choices.txt"
output_table_path = "C:/Users/Arkady/Dropbox/Research/Journal papers/2019 Changes-of-mind/tables/"

choice.data = read.table(data_path, sep="\t", header=T)

A bit of preprocessing to create variables for analyzing sequential effects

In [6]:
preprocess_data <- function(choice.data) {
    # any exclusions?
    stats.df = choice.data
    stats.df$subj_id = as.factor(stats.df$subj_id)

    # generate trial number
    max.trial = max(stats.df$trial_no)
    max.block = max(stats.df$block_n)
    stats.df$all_trial_no = (stats.df$session_no-1) * (max.block * max.trial) +
      (stats.df$block_no-1) * max.trial +
      stats.df$trial_no

    contrasts(stats.df$is_com) = contr.sum(n=2) /2 *-1
    contrasts(stats.df$is_com)# true is pos
    
#     stats.df = stats.df[(stats.df$RT < 1.5),]

    str(stats.df)

    return(stats.df)
}

stats.df = preprocess_data(choice.data)

'data.frame':	9564 obs. of  28 variables:
 $ subj_id        : Factor w/ 4 levels "269","275","391",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ session_no     : int  1 1 1 1 1 1 1 1 1 1 ...
 $ block_no       : int  1 1 1 1 1 1 1 1 1 1 ...
 $ trial_no       : int  1 2 3 4 5 6 7 8 9 10 ...
 $ is_practice    : Factor w/ 2 levels "False","True": 2 2 2 2 2 2 2 2 2 2 ...
 $ direction      : num  0 180 0 0 0 180 180 0 0 180 ...
 $ coherence      : num  0.512 0 0.064 0.064 0.064 0.032 0.512 0.032 0.256 0.128 ...
 $ duration       : int  800 800 800 800 800 800 800 800 800 800 ...
 $ response       : int  0 180 180 180 0 0 180 0 0 180 ...
 $ trial_time     : num  3.1 3.5 1.14 2.54 1.05 ...
 $ is_correct     : Factor w/ 2 levels "False","True": 2 2 1 1 2 1 2 2 2 2 ...
 $ xflips         : int  1 1 0 1 0 0 0 0 0 0 ...
 $ max_d          : num  139 116 -48.9 35.2 193.7 ...
 $ idx_max_d      : num  224 284 83 209 72 172 64 107 130 204 ...
 $ midline_d      : num  1 0 0 1 0 0 0 0 0 0 ...
 $ idx_midline_d  : num  20

# Analysis 1. Accuracy as a function of coherence

In [7]:
rnd_effects_analysis_1 <- function(stats.df){
    rnd1.lmer = glmer(is_correct ~ (1|subj_id), 
                 stats.df[stats.df$coherence!=0,], # edit this line to include zero coh
                 family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff learning effect for each p
    rnd2.lmer = glmer(is_correct ~ (c.(l.(all_trial_no))|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd intercept for each participant and random slope of coherence
    # diff avg acc, diff coherence effect for each p
    rnd3.lmer = glmer(is_correct ~ (c.(coherence)|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff linear and quad learning effect for each p
    rnd4.lmer = glmer(is_correct ~ (poly(coherence, 2, raw = T)|subj_id), 
                           stats.df[stats.df$coherence!=0,], 
                           family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff trial-to-trial stimulus effect and diff learning effect for each p
    rnd5.lmer = glmer(is_correct ~ (c.(l.(all_trial_no)) + c.(coherence)|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd.lmer with com and coherence
    rnd6.lmer = glmer(is_correct ~ ((is_com + c.(coherence))|subj_id),
                      stats.df[stats.df$coherence!=0,],
                      family = binomial)

    rnd.anova = anova(rnd1.lmer, rnd2.lmer, rnd3.lmer, rnd4.lmer, rnd5.lmer, rnd6.lmer)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [8]:
rnd_effects_analysis_1(stats.df)

singular fit
singular fit
singular fit
singular fit


Data: stats.df[stats.df$coherence != 0, ]
Models:
rnd1.lmer: is_correct ~ (1 | subj_id)
rnd2.lmer: is_correct ~ (c.(l.(all_trial_no)) | subj_id)
rnd3.lmer: is_correct ~ (c.(coherence) | subj_id)
rnd4.lmer: is_correct ~ (poly(coherence, 2, raw = T) | subj_id)
rnd5.lmer: is_correct ~ (c.(l.(all_trial_no)) + c.(coherence) | subj_id)
rnd6.lmer: is_correct ~ ((is_com + c.(coherence)) | subj_id)
          Df    AIC    BIC  logLik deviance     Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  2 9049.7 9063.6 -4522.8   9045.7                                
rnd2.lmer  4 9053.5 9081.4 -4522.8   9045.5    0.1718      2     0.9177    
rnd3.lmer  4 7603.0 7631.0 -3797.5   7595.0 1450.4796      0     <2e-16 ***
rnd4.lmer  7 7607.9 7656.8 -3797.0   7593.9    1.1211      3     0.7720    
rnd5.lmer  7 7606.4 7655.3 -3796.2   7592.4    1.4747      0     <2e-16 ***
rnd6.lmer  7 7614.5 7663.4 -3800.3   7600.5    0.0000      0     1.0000    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Bes

rnd3 is the best

In [82]:
run_analysis_1 <- function(stats.df){
    choice.mer = glmer(is_correct ~ ((coherence | subj_id)) + c.(coherence)*is_com + c.(l.(all_trial_no)),
                      stats.df[stats.df$coherence!=0,],
                      family = binomial)
    print(summary(choice.mer))

    choice.output = summary(choice.mer)$coefficients
    row.names(choice.output) <- c("Intercept", "Coherence", "Is CoM", "Trial number", "Coherence by Is CoM")

    file_name = paste(output_table_path, "is_correct_vs_coh.tex", sep="")
    print(xtable(choice.output, digits = c(4,4,4,4,4),# display = c("g","g","g","g","g"), 
                 label = "tab:is_correct_vs_coh",
                 caption = "Parameters of a linear mixed-effects model analysing choice accuracy 
                            as a function of coherence, presence or absence of a change-of-mind, and trial number."), 
          caption.placement = "top", table.placement="t", floating.environment = "table*",
          math.style.exponents = TRUE, type = "latex", booktabs = TRUE, file = file_name)
}

In [83]:
run_analysis_1(stats.df)

singular fit


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_correct ~ ((coherence | subj_id)) + c.(coherence) * is_com +  
    c.(l.(all_trial_no))
   Data: stats.df[stats.df$coherence != 0, ]

     AIC      BIC   logLik deviance df.resid 
  7579.1   7635.0  -3781.6   7563.1     7965 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-17.4904  -0.9695   0.2526   0.6938   1.1790 

Random effects:
 Groups  Name        Variance Std.Dev. Corr 
 subj_id (Intercept) 0.001469 0.03833       
         coherence   5.620370 2.37073  -1.00
Number of obs: 7973, groups:  subj_id, 4

Fixed effects:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)            1.14320    0.24861   4.598 4.26e-06 ***
c.(coherence)          7.01297    1.52191   4.608 4.07e-06 ***
is_com1               -1.03722    0.25467  -4.073 4.64e-05 ***
c.(l.(all_trial_no))   0.04911    0.02792   1.759   0.0786 .  
c.(

# Analysis 2. Probability of CoM as a function of initiation time

In [33]:
rnd_effects_analysis_2 <- function(stats.df){
    rnd1.lmer = glmer(is_com ~ (1|subj_id), stats.df, family = binomial)

    rnd2.lmer = glmer(is_com ~ (c.(coherence)|subj_id), stats.df, family = binomial)

    rnd3.lmer = glmer(is_com ~ (poly(c.(coherence), 2, raw = T)|subj_id), stats.df, family = binomial)

    rnd4.lmer = glmer(is_com ~ (c.(RT)|subj_id), stats.df, family = binomial)
    
    rnd5.lmer = glmer(is_com ~ (c.(RT)*c.(coherence)|subj_id), stats.df, family = binomial)
    
    rnd.anova = anova(rnd1.lmer, rnd2.lmer, rnd3.lmer, rnd4.lmer, rnd5.lmer)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [21]:
rnd_effects_analysis_2(stats.df)

singular fit
singular fit
singular fit


Data: stats.df
Models:
rnd1.lmer: is_com ~ (1 | subj_id)
rnd2.lmer: is_com ~ (c.(coherence) | subj_id)
rnd4.lmer: is_com ~ (c.(RT) | subj_id)
rnd3.lmer: is_com ~ (poly(c.(coherence), 2, raw = T) | subj_id)
rnd5.lmer: is_com ~ (c.(RT) * c.(coherence) | subj_id)
          Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  2 2324.1 2338.5 -1160.1   2320.1                             
rnd2.lmer  4 2255.3 2283.9 -1123.6   2247.3 72.843      2     <2e-16 ***
rnd4.lmer  4 2187.8 2216.5 -1089.9   2179.8 67.456      0     <2e-16 ***
rnd3.lmer  7 2240.9 2291.1 -1113.5   2226.9  0.000      3          1    
rnd5.lmer 11 2167.7 2246.5 -1072.8   2145.7 81.243      4     <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd5.lmer"
[1] "Best model according to BIC"
[1] "rnd4.lmer"


rnd4 is the best converging model

In [84]:
run_analysis_2 <- function(stats.df, exp_name){
    choice.mer = glmer(is_com ~ (c.(RT) | subj_id) + c.(coherence)*c.(RT), stats.df, family = binomial)
    print(summary(choice.mer))

    choice.output = summary(choice.mer)$coefficients
    row.names(choice.output) <- c("Intercept", "Coherence", "RT", "Coherence by RT")

    file_name = paste(output_table_path, "is_com_vs_RT.tex", sep="")
    print(xtable(choice.output, digits = c(4,4,4,4,4),
                 label = "tab:is_com_vs_RT",
                 caption = "Parameters of a linear mixed-effects model analysing probability of a change-of-mind 
                            as a function of coherence and response time."), 
          caption.placement = "top", table.placement="t", floating.environment = "table*",
          math.style.exponents = TRUE, type = "latex", booktabs = TRUE, file = file_name)
}

In [85]:
run_analysis_2(stats.df)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_com ~ (c.(RT) | subj_id) + c.(coherence) * c.(RT)
   Data: stats.df

     AIC      BIC   logLik deviance df.resid 
  2156.8   2207.0  -1071.4   2142.8     9557 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.3986 -0.1992 -0.1070 -0.0578 22.4107 

Random effects:
 Groups  Name        Variance Std.Dev. Corr 
 subj_id (Intercept) 1.447    1.203         
         c.(RT)      1.170    1.081    -0.01
Number of obs: 9564, groups:  subj_id, 4

Fixed effects:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)           -4.3238     0.6156  -7.024 2.16e-12 ***
c.(coherence)         -2.7840     0.6161  -4.519 6.23e-06 ***
c.(RT)                 1.4052     0.5945   2.364   0.0181 *  
c.(coherence):c.(RT)   2.5390     0.9415   2.697   0.0070 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of 

# Analysis 3. Response time as a function of coherence

In [33]:
rnd_effects_analysis_3 <- function(stats.df){
#     stats.df <- stats.df[complete.cases(stats.df[ , "eye_IT"]),]
    rnd1.lmer = lmer(RT ~ (1|subj_id), stats.df)

    rnd2.lmer = lmer(RT ~ (c.(coherence)|subj_id), stats.df)

    rnd3.lmer = lmer(RT ~ (is_correct|subj_id), stats.df)

    rnd4.lmer = lmer(RT ~ ((c.(coherence)+is_correct)|subj_id), stats.df)
    
    rnd5.lmer = lmer(RT ~ ((c.(coherence)*is_correct)|subj_id), stats.df)
    
    rnd.anova = anova(rnd1.lmer, rnd2.lmer, rnd3.lmer, rnd4.lmer, rnd5.lmer)
#     rnd.anova = anova(rnd4.lmer, rnd5.lmer)
    print(rnd.anova)
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [35]:
rnd_effects_analysis_3(stats.df)

singular fit
singular fit
singular fit
singular fit
refitting model(s) with ML (instead of REML)


Data: stats.df
Models:
rnd1.lmer: RT ~ (1 | subj_id)
rnd2.lmer: RT ~ (c.(coherence) | subj_id)
rnd3.lmer: RT ~ (is_correct | subj_id)
rnd4.lmer: RT ~ ((c.(coherence) + is_correct) | subj_id)
rnd5.lmer: RT ~ ((c.(coherence) * is_correct) | subj_id)
          Df     AIC     BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  3 12293.6 12315.1 -6143.8  12287.6                             
rnd2.lmer  5 10001.8 10037.6 -4995.9   9991.8 2295.8      2  < 2.2e-16 ***
rnd3.lmer  5 11572.2 11608.0 -5781.1  11562.2    0.0      0          1    
rnd4.lmer  8  9880.4  9937.7 -4932.2   9864.4 1697.8      3  < 2.2e-16 ***
rnd5.lmer 12  9827.8  9913.8 -4901.9   9803.8   60.6      4   2.17e-12 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd5.lmer"
[1] "Best model according to BIC"
[1] "rnd5.lmer"


rnd5 is the best

In [86]:
run_analysis_3 <- function(stats.df, exp_name){
    choice.mer = lmer(RT ~ ((c.(coherence)*is_correct) | subj_id) + c.(coherence)*is_correct, stats.df)
    print(summary(choice.mer))

    choice.output = summary(choice.mer)$coefficients

    row.names(choice.output) <- c("Intercept", "Coherence", "Is correct", "Coherence by Is Correct")

    file_name = paste(output_table_path, "RT_vs_coh.tex", sep="")
    print(xtable(choice.output, digits = c(4,4,4,4,4,4),
                 label = "tab:RT_vs_coh",
                 caption = "Parameters of a linear mixed-effects model analysing response time as
                            a function of coherence and choice accuracy."), 
          caption.placement = "top", table.placement="t", floating.environment = "table*",
          math.style.exponents = TRUE, type = "latex", booktabs = TRUE, file = file_name)
}

In [87]:
run_analysis_3(stats.df)

singular fit


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: RT ~ ((c.(coherence) * is_correct) | subj_id) + c.(coherence) *  
    is_correct
   Data: stats.df

REML criterion at convergence: 9811.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.3680 -0.4716 -0.1189  0.2616  7.3463 

Random effects:
 Groups   Name                         Variance Std.Dev. Corr             
 subj_id  (Intercept)                  0.047198 0.21725                   
          c.(coherence)                0.660265 0.81257  -0.94            
          is_correctTrue               0.005195 0.07208  -0.97  0.82      
          c.(coherence):is_correctTrue 0.033216 0.18225  -0.34 -0.01  0.57
 Residual                              0.162583 0.40322                   
Number of obs: 9564, groups:  subj_id, 4

Fixed effects:
                             Estimate Std. Error       df t value Pr(>|t|)   
(Intercept)                   0.52133    0.10939  2.98221   4.7